In [1]:
import numpy as np

In [2]:
def get_error(x, y):
    error = x - y
    print( np.linalg.norm(error.flatten()) )

In [3]:
from pyscf import gto, scf

from ccpy.models.calculation import Calculation
from ccpy.interfaces.pyscf_tools import load_pyscf_integrals
from ccpy.drivers.driver import cc_driver

mol = gto.Mole()
mol.build(
    atom="""F 0.0 0.0 -2.66816
            F 0.0 0.0  2.66816""",
    basis="ccpvdz",
    charge=0,
    spin=0,
    symmetry="D2H",
    cart=True,
    unit='Bohr',
)
mf = scf.ROHF(mol)
mf.kernel()


system, H0 = load_pyscf_integrals(mf, nfrozen=2)
system.print_info()

converged SCF energy = -198.420096282673
   System Information:
   ----------------------------------------------------
     Number of correlated electrons = 14
     Number of correlated orbitals = 28
     Number of frozen orbitals = 2
     Number of alpha occupied orbitals = 7
     Number of alpha unoccupied orbitals = 21
     Number of beta occupied orbitals = 7
     Number of beta unoccupied orbitals = 21
     Charge = 0
     Point group = D2h
     Symmetry of reference = AG
     Spin multiplicity of reference = 1

      MO #        Energy (a.u.)      Symmetry    Occupation
-----------------------------------------------------------
         1           -26.443066            AG           2.0
         2           -26.443033           B1U           2.0
         3            -1.597219            AG           2.0
         4            -1.589112           B1U           2.0
         5            -0.725614           B2U           2.0
         6            -0.725614           B3U           

In [4]:
calculation = Calculation(
    order=3,
    active_orders=[None],
    num_active=[None],
    calculation_type="ccsdt",
    convergence_tolerance=1.0e-08
)

T0, total_energy, is_converged = cc_driver(calculation, system, H0)

   --------------------------------------------------
   Calculation type =  CCSDT
   Maximum iterations = 60
   Convergence tolerance = 1e-08
   Energy shift = 0.0
   DIIS size =  6
   RHF symmetry = False
   --------------------------------------------------

   CC calculation started at 2022-03-10 09:28:00 

   Energy of initial guess =         0.0000000000

                 Iter.         Residuum                 δE                  ΔE               Wall time
-----------------------------------------------------------------------------------------------------
                   0         0.7950811421        -0.5760307223        -0.5760307223          (0.0m 6.6s)
                   1         0.2600010113        -0.0470790932        -0.6231098155          (0.0m 7.8s)
                   2         0.0673753279        -0.0073211685        -0.6304309840         (0.0m 10.7s)
                   3         0.0268057665        -0.0056287055        -0.6360596895          (0.0m 9.3s)
           

In [5]:
from ccpy.hbar.hbar_ccsd import build_hbar_ccsd

H = build_hbar_ccsd(T0, H0)

In [6]:
noccupied_alpha = T0.a.shape[1]
nunoccupied_alpha = T0.a.shape[0]
noccupied_beta = T0.b.shape[1]
nunoccupied_beta = T0.b.shape[0]

nact_occupied_alpha = 4
nact_unoccupied_alpha = 10
nact_occupied_beta = 4
nact_unoccupied_beta = 10

nunact_occupied_alpha = noccupied_alpha - nact_occupied_alpha
nunact_unoccupied_alpha = nunoccupied_alpha - nact_unoccupied_alpha
nunact_occupied_beta = noccupied_beta - nact_occupied_beta
nunact_unoccupied_beta = nunoccupied_beta - nact_unoccupied_beta

oa = slice(0, noccupied_alpha - nact_occupied_alpha)
Oa = slice(noccupied_alpha - nact_occupied_alpha, noccupied_alpha)
va = slice(nact_unoccupied_alpha, nunoccupied_alpha)
Va = slice(0, nact_unoccupied_alpha)

ob = slice(0, noccupied_beta - nact_occupied_beta)
Ob = slice(noccupied_beta - nact_occupied_beta, noccupied_beta)
vb = slice(nact_unoccupied_beta, nunoccupied_beta)
Vb = slice(0, nact_unoccupied_beta)

shift = 0.0

In [7]:
# An additional step needs to be done. We need to zero all T3 amplitudes 
# outside of the active space before computing the full CCSDT updates.
# This is because the active-space CC updates assume that T3 is 0 outside of
# the active space. If this is not true in the full scheme, then the results
# will not match.

def active_hole(x, nocc, nact):
    if x < nocc - nact:
        return 0
    else:
        return 1
    
def active_particle(x, nact):
    if x < nact:
        return 1
    else:
        return 0
    
# assume active spaces for alpha and beta are the same for now
def is_active(a, b, c, i, j, k, nocc, nact_h, nact_p):
    h = 0
    p = 0
    for n in [i, j, k]:
        h += active_hole(n, nocc, nact_h)
    for n in [a, b, c]:
        p += active_particle(n, nact_p)
    if h >= 1 and p >=1:
        return True
    else:
        return False


for a in range(system.nunoccupied_alpha):
    for b in range(a+1, system.nunoccupied_alpha):
        for c in range(b+1, system.nunoccupied_alpha):
            for i in range(system.noccupied_alpha):
                for j in range(i+1, system.noccupied_alpha):
                    for k in range(j+1, system.noccupied_alpha):
                        if not is_active(a, b, c, i, j, k, 
                                    system.noccupied_alpha, 
                                    nact_occupied_alpha, nact_unoccupied_alpha):
                            T0.aaa[a, b, c, i, j, k] = 0.0
                            T0.aaa[a, b, c, j, i, k] = 0.0
                            T0.aaa[a, b, c, j, k, i] = 0.0
                            T0.aaa[a, b, c, i, k, j] = 0.0
                            T0.aaa[a, b, c, k, j, i] = 0.0
                            T0.aaa[a, b, c, k, i, j] = 0.0
                            
                            T0.aaa[a, c, b, i, j, k] = 0.0
                            T0.aaa[a, c, b, j, i, k] = 0.0
                            T0.aaa[a, c, b, j, k, i] = 0.0
                            T0.aaa[a, c, b, i, k, j] = 0.0
                            T0.aaa[a, c, b, k, j, i] = 0.0
                            T0.aaa[a, c, b, k, i, j] = 0.0
                            
                            T0.aaa[b, a, c, i, j, k] = 0.0
                            T0.aaa[b, a, c, j, i, k] = 0.0
                            T0.aaa[b, a, c, j, k, i] = 0.0
                            T0.aaa[b, a, c, i, k, j] = 0.0
                            T0.aaa[b, a, c, k, j, i] = 0.0
                            T0.aaa[b, a, c, k, i, j] = 0.0
                            
                            T0.aaa[b, c, a, i, j, k] = 0.0
                            T0.aaa[b, c, a, j, i, k] = 0.0
                            T0.aaa[b, c, a, j, k, i] = 0.0
                            T0.aaa[b, c, a, i, k, j] = 0.0
                            T0.aaa[b, c, a, k, j, i] = 0.0
                            T0.aaa[b, c, a, k, i, j] = 0.0
                            
                            T0.aaa[c, a, b, i, j, k] = 0.0
                            T0.aaa[c, a, b, j, i, k] = 0.0
                            T0.aaa[c, a, b, j, k, i] = 0.0
                            T0.aaa[c, a, b, i, k, j] = 0.0
                            T0.aaa[c, a, b, k, j, i] = 0.0
                            T0.aaa[c, a, b, k, i, j] = 0.0
                            
                            T0.aaa[c, b, a, i, j, k] = 0.0
                            T0.aaa[c, b, a, j, i, k] = 0.0
                            T0.aaa[c, b, a, j, k, i] = 0.0
                            T0.aaa[c, b, a, i, k, j] = 0.0
                            T0.aaa[c, b, a, k, j, i] = 0.0
                            T0.aaa[c, b, a, k, i, j] = 0.0
                            
                            
for a in range(system.nunoccupied_alpha):
    for b in range(a+1, system.nunoccupied_alpha):
        for c in range(system.nunoccupied_beta):
            for i in range(system.noccupied_alpha):
                for j in range(i+1, system.noccupied_alpha):
                    for k in range(system.noccupied_beta):
                        if not is_active(a, b, c, i, j, k, 
                                    system.noccupied_alpha, 
                                    nact_occupied_alpha, nact_unoccupied_alpha):
                            T0.aab[a, b, c, i, j, k] = 0.0
                            T0.aab[b, a, c, i, j, k] = 0.0
                            T0.aab[a, b, c, j, i, k] = 0.0
                            T0.aab[b, a, c, j, i, k] = 0.0
                            
for a in range(system.nunoccupied_alpha):
    for b in range(system.nunoccupied_beta):
        for c in range(b+1, system.nunoccupied_beta):
            for i in range(system.noccupied_alpha):
                for j in range(system.noccupied_beta ):
                    for k in range(j+1, system.noccupied_beta):
                        if not is_active(a, b, c, i, j, k, 
                                    system.noccupied_alpha, 
                                    nact_occupied_alpha, nact_unoccupied_alpha):
                            T0.abb[a, b, c, i, j, k] = 0.0
                            T0.abb[b, a, c, i, j, k] = 0.0
                            T0.abb[a, b, c, j, i, k] = 0.0
                            T0.abb[b, a, c, j, i, k] = 0.0

In [8]:
# Populating the active-space structure
from ccpy.models.operators import ClusterOperator

T = ClusterOperator(system, 3, active_orders=[3], num_active=[1])

# T.aaa
T.aaa.VVVOOO = T0.aaa[Va, Va, Va, Oa, Oa, Oa]
T.aaa.VVvOOO = T0.aaa[Va, Va, va, Oa, Oa, Oa]
T.aaa.VVVoOO = T0.aaa[Va, Va, Va, oa, Oa, Oa]
T.aaa.VVvoOO = T0.aaa[Va, Va, va, oa, Oa, Oa]
T.aaa.VvvOOO = T0.aaa[Va, va, va, Oa, Oa, Oa]
T.aaa.VVVooO = T0.aaa[Va, Va, Va, oa, oa, Oa]
T.aaa.VvvoOO = T0.aaa[Va, va, va, oa, Oa, Oa]
T.aaa.VVvooO = T0.aaa[Va, Va, va, oa, oa, Oa]
T.aaa.VvvooO = T0.aaa[Va, va, va, oa, oa, Oa]

# T.aab
T.aab.VVVOOO = T0.aab[Va,Va,Vb,Oa,Oa,Ob]
T.aab.VVVoOO = T0.aab[Va,Va,Vb,oa,Oa,Ob]
T.aab.VVVOOo = T0.aab[Va,Va,Vb,Oa,Oa,ob]
T.aab.VVVoOo = T0.aab[Va,Va,Vb,oa,Oa,ob]
T.aab.VVVooO = T0.aab[Va,Va,Vb,oa,oa,Ob]

T.aab.VvVOOO = T0.aab[Va,va,Vb,Oa,Oa,Ob]
T.aab.VvVoOO = T0.aab[Va,va,Vb,oa,Oa,Ob]
T.aab.VvVOOo = T0.aab[Va,va,Vb,Oa,Oa,ob]
T.aab.VvVoOo = T0.aab[Va,va,Vb,oa,Oa,ob]
T.aab.VvVooO = T0.aab[Va,va,Vb,oa,oa,Ob]

T.aab.VVvOOO = T0.aab[Va,Va,vb,Oa,Oa,Ob]
T.aab.VVvoOO = T0.aab[Va,Va,vb,oa,Oa,Ob]
T.aab.VVvOOo = T0.aab[Va,Va,vb,Oa,Oa,ob]
T.aab.VVvoOo = T0.aab[Va,Va,vb,oa,Oa,ob]
T.aab.VVvooO = T0.aab[Va,Va,vb,oa,oa,Ob]

T.aab.VvvOOO = T0.aab[Va,va,vb,Oa,Oa,Ob]
T.aab.VvvoOO = T0.aab[Va,va,vb,oa,Oa,Ob]
T.aab.VvvOOo = T0.aab[Va,va,vb,Oa,Oa,ob]
T.aab.VvvoOo = T0.aab[Va,va,vb,oa,Oa,ob]
T.aab.VvvooO = T0.aab[Va,va,vb,oa,oa,Ob]

T.aab.vvVOOO = T0.aab[va,va,Vb,Oa,Oa,Ob]
T.aab.vvVoOO = T0.aab[va,va,Vb,oa,Oa,Ob]
T.aab.vvVOOo = T0.aab[va,va,Vb,Oa,Oa,ob]
T.aab.vvVoOo = T0.aab[va,va,Vb,oa,Oa,ob]
T.aab.vvVooO = T0.aab[va,va,Vb,oa,oa,Ob]

# T.abb
T.abb.VVVOOO = T0.abb[Va, Vb, Vb, Oa, Ob, Ob]
T.abb.VVVoOO = T0.abb[Va, Vb, Vb, oa, Ob, Ob]
T.abb.VVVOoO = T0.abb[Va, Vb, Vb, Oa, ob, Ob]
T.abb.VVVooO = T0.abb[Va, Vb, Vb, oa, ob, Ob]
T.abb.VVVOoo = T0.abb[Va, Vb, Vb, Oa, ob, ob]

T.abb.vVVOOO = T0.abb[va, Vb, Vb, Oa, Ob, Ob]
T.abb.vVVoOO = T0.abb[va, Vb, Vb, oa, Ob, Ob]
T.abb.vVVOoO = T0.abb[va, Vb, Vb, Oa, ob, Ob]
T.abb.vVVooO = T0.abb[va, Vb, Vb, oa, ob, Ob]
T.abb.vVVOoo = T0.abb[va, Vb, Vb, Oa, ob, ob]

T.abb.VVvOOO = T0.abb[Va, Vb, vb, Oa, Ob, Ob]
T.abb.VVvoOO = T0.abb[Va, Vb, vb, oa, Ob, Ob]
T.abb.VVvOoO = T0.abb[Va, Vb, vb, Oa, ob, Ob]
T.abb.VVvooO = T0.abb[Va, Vb, vb, oa, ob, Ob]
T.abb.VVvOoo = T0.abb[Va, Vb, vb, Oa, ob, ob]

T.abb.vVvOOO = T0.abb[va, Vb, vb, Oa, Ob, Ob]
T.abb.vVvoOO = T0.abb[va, Vb, vb, oa, Ob, Ob]
T.abb.vVvOoO = T0.abb[va, Vb, vb, Oa, ob, Ob]
T.abb.vVvooO = T0.abb[va, Vb, vb, oa, ob, Ob]
T.abb.vVvOoo = T0.abb[va, Vb, vb, Oa, ob, ob]

T.abb.VvvOOO = T0.abb[Va, vb, vb, Oa, Ob, Ob]
T.abb.VvvoOO = T0.abb[Va, vb, vb, oa, Ob, Ob]
T.abb.VvvOoO = T0.abb[Va, vb, vb, Oa, ob, Ob]
T.abb.VvvooO = T0.abb[Va, vb, vb, oa, ob, Ob]
T.abb.VvvOoo = T0.abb[Va, vb, vb, Oa, ob, ob]


In [9]:
from ccpy.utilities.updates import cc_loops2
from ccpy.utilities.updates import cc_active_loops

In [10]:
X3B = -0.5 * np.einsum("mi,abcmjk->abcijk", H.a.oo, T0.aab, optimize=True)
X3B -= 0.25 * np.einsum("mk,abcijm->abcijk", H.b.oo, T0.aab, optimize=True)
X3B += 0.5 * np.einsum("ae,ebcijk->abcijk", H.a.vv, T0.aab, optimize=True)
X3B += 0.25 * np.einsum("ce,abeijk->abcijk", H.b.vv, T0.aab, optimize=True)
X3B += 0.125 * np.einsum("mnij,abcmnk->abcijk", H.aa.oooo, T0.aab, optimize=True)
X3B += 0.5 * np.einsum("mnjk,abcimn->abcijk", H.ab.oooo, T0.aab, optimize=True)
X3B += 0.125 * np.einsum("abef,efcijk->abcijk", H.aa.vvvv, T0.aab, optimize=True)
X3B += 0.5 * np.einsum("bcef,aefijk->abcijk", H.ab.vvvv, T0.aab, optimize=True)
X3B += np.einsum("amie,ebcmjk->abcijk", H.aa.voov, T0.aab, optimize=True)
X3B += np.einsum("amie,becjmk->abcijk", H.ab.voov, T0.abb, optimize=True)
X3B += 0.25 * np.einsum("mcek,abeijm->abcijk", H.ab.ovvo, T0.aaa, optimize=True)
X3B += 0.25 * np.einsum("cmke,abeijm->abcijk", H.bb.voov, T0.aab, optimize=True)
X3B -= 0.5 * np.einsum("amek,ebcijm->abcijk", H.ab.vovo, T0.aab, optimize=True)
X3B -= 0.5 * np.einsum("mcie,abemjk->abcijk", H.ab.ovov, T0.aab, optimize=True)

X3B -= np.transpose(X3B, (1, 0, 2, 3, 4, 5))
X3B -= np.transpose(X3B, (0, 1, 2, 4, 3, 5))

## Projection 1

In [11]:
x3b = (2.0/4.0) * (
        +1.0*np.einsum('mI,BACmJK->ABCIJK', H.a.oo[oa, Oa], T.aab.VVVoOO, optimize=True)
        +1.0*np.einsum('MI,BACMJK->ABCIJK', H.a.oo[Oa, Oa], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        +1.0*np.einsum('mK,BACIJm->ABCIJK', H.b.oo[ob, Ob], T.aab.VVVOOo, optimize=True)
        +1.0*np.einsum('MK,BACIJM->ABCIJK', H.b.oo[Ob, Ob], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        -1.0*np.einsum('Ae,BeCIJK->ABCIJK', H.a.vv[Va, va], T.aab.VvVOOO, optimize=True)
        -1.0*np.einsum('AE,BECIJK->ABCIJK', H.a.vv[Va, Va], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        -1.0*np.einsum('Ce,BAeIJK->ABCIJK', H.b.vv[Vb, vb], T.aab.VVvOOO, optimize=True)
        -1.0*np.einsum('CE,BAEIJK->ABCIJK', H.b.vv[Vb, Vb], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        -0.5*np.einsum('mnIJ,BACmnK->ABCIJK', H.aa.oooo[oa, oa, Oa, Oa], T.aab.VVVooO, optimize=True)
        -1.0*np.einsum('mNIJ,BACmNK->ABCIJK', H.aa.oooo[oa, Oa, Oa, Oa], T.aab.VVVoOO, optimize=True)
        -0.5*np.einsum('MNIJ,BACMNK->ABCIJK', H.aa.oooo[Oa, Oa, Oa, Oa], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('mnJK,BACmIn->ABCIJK', H.ab.oooo[oa, ob, Oa, Ob], T.aab.VVVoOo, optimize=True)
        -1.0*np.einsum('MnJK,BACIMn->ABCIJK', H.ab.oooo[Oa, ob, Oa, Ob], T.aab.VVVOOo, optimize=True)
        +1.0*np.einsum('mNJK,BACmIN->ABCIJK', H.ab.oooo[oa, Ob, Oa, Ob], T.aab.VVVoOO, optimize=True)
        -1.0*np.einsum('MNJK,BACIMN->ABCIJK', H.ab.oooo[Oa, Ob, Oa, Ob], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        -0.5*np.einsum('ABef,feCIJK->ABCIJK', H.aa.vvvv[Va, Va, va, va], T.aab.vvVOOO, optimize=True)
        -1.0*np.einsum('ABeF,FeCIJK->ABCIJK', H.aa.vvvv[Va, Va, va, Va], T.aab.VvVOOO, optimize=True)
        -0.5*np.einsum('ABEF,FECIJK->ABCIJK', H.aa.vvvv[Va, Va, Va, Va], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('BCef,AefIJK->ABCIJK', H.ab.vvvv[Va, Vb, va, vb], T.aab.VvvOOO, optimize=True)
        -1.0*np.einsum('BCEf,EAfIJK->ABCIJK', H.ab.vvvv[Va, Vb, Va, vb], T.aab.VVvOOO, optimize=True)
        +1.0*np.einsum('BCeF,AeFIJK->ABCIJK', H.ab.vvvv[Va, Vb, va, Vb], T.aab.VvVOOO, optimize=True)
        -1.0*np.einsum('BCEF,EAFIJK->ABCIJK', H.ab.vvvv[Va, Vb, Va, Vb], T.aab.VVVOOO, optimize=True)
)
x3b += (4.0/4.0) * (
        -1.0*np.einsum('AmIe,BeCmJK->ABCIJK', H.aa.voov[Va, oa, Oa, va], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('AmIE,BECmJK->ABCIJK', H.aa.voov[Va, oa, Oa, Va], T.aab.VVVoOO, optimize=True)
        -1.0*np.einsum('AMIe,BeCMJK->ABCIJK', H.aa.voov[Va, Oa, Oa, va], T.aab.VvVOOO, optimize=True)
        -1.0*np.einsum('AMIE,BECMJK->ABCIJK', H.aa.voov[Va, Oa, Oa, Va], T.aab.VVVOOO, optimize=True)
)
x3b += (4.0/4.0) * (
        -1.0*np.einsum('AmIe,BCeJmK->ABCIJK', H.ab.voov[Va, ob, Oa, vb], T.abb.VVvOoO, optimize=True)
        -1.0*np.einsum('AmIE,BCEJmK->ABCIJK', H.ab.voov[Va, ob, Oa, Vb], T.abb.VVVOoO, optimize=True)
        -1.0*np.einsum('AMIe,BCeJMK->ABCIJK', H.ab.voov[Va, Ob, Oa, vb], T.abb.VVvOOO, optimize=True)
        -1.0*np.einsum('AMIE,BCEJMK->ABCIJK', H.ab.voov[Va, Ob, Oa, Vb], T.abb.VVVOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        -1.0*np.einsum('mCeK,BAemIJ->ABCIJK', H.ab.ovvo[oa, Vb, va, Ob], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('mCEK,EBAmIJ->ABCIJK', H.ab.ovvo[oa, Vb, Va, Ob], T.aaa.VVVoOO, optimize=True)
        -1.0*np.einsum('MCeK,BAeIJM->ABCIJK', H.ab.ovvo[Oa, Vb, va, Ob], T.aaa.VVvOOO, optimize=True)
        -1.0*np.einsum('MCEK,EBAIJM->ABCIJK', H.ab.ovvo[Oa, Vb, Va, Ob], T.aaa.VVVOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        -1.0*np.einsum('CmKe,BAeIJm->ABCIJK', H.bb.voov[Vb, ob, Ob, vb], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('CmKE,BAEIJm->ABCIJK', H.bb.voov[Vb, ob, Ob, Vb], T.aab.VVVOOo, optimize=True)
        -1.0*np.einsum('CMKe,BAeIJM->ABCIJK', H.bb.voov[Vb, Ob, Ob, vb], T.aab.VVvOOO, optimize=True)
        -1.0*np.einsum('CMKE,BAEIJM->ABCIJK', H.bb.voov[Vb, Ob, Ob, Vb], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('AmeK,BeCIJm->ABCIJK', H.ab.vovo[Va, ob, va, Ob], T.aab.VvVOOo, optimize=True)
        +1.0*np.einsum('AmEK,BECIJm->ABCIJK', H.ab.vovo[Va, ob, Va, Ob], T.aab.VVVOOo, optimize=True)
        +1.0*np.einsum('AMeK,BeCIJM->ABCIJK', H.ab.vovo[Va, Ob, va, Ob], T.aab.VvVOOO, optimize=True)
        +1.0*np.einsum('AMEK,BECIJM->ABCIJK', H.ab.vovo[Va, Ob, Va, Ob], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('mCIe,BAemJK->ABCIJK', H.ab.ovov[oa, Vb, Oa, vb], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('mCIE,BAEmJK->ABCIJK', H.ab.ovov[oa, Vb, Oa, Vb], T.aab.VVVoOO, optimize=True)
        +1.0*np.einsum('MCIe,BAeMJK->ABCIJK', H.ab.ovov[Oa, Vb, Oa, vb], T.aab.VVvOOO, optimize=True)
        +1.0*np.einsum('MCIE,BAEMJK->ABCIJK', H.ab.ovov[Oa, Vb, Oa, Vb], T.aab.VVVOOO, optimize=True)
)


x3b -= np.transpose(x3b, (1, 0, 2, 3, 4, 5))
x3b -= np.transpose(x3b, (0, 1, 2, 4, 3, 5))

get_error(X3B[Va, Va, Vb, Oa, Oa, Ob], x3b)

8.49104622295171e-17


### Projection 2

In [12]:
x3b = (2.0/4.0) * (
        +1.0*np.einsum('mI,BAcmJK->ABcIJK', H.a.oo[oa, Oa], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('MI,BAcMJK->ABcIJK', H.a.oo[Oa, Oa], T.aab.VVvOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        +1.0*np.einsum('mK,BAcIJm->ABcIJK', H.b.oo[ob, Ob], T.aab.VVvOOo, optimize=True)
        +1.0*np.einsum('MK,BAcIJM->ABcIJK', H.b.oo[Ob, Ob], T.aab.VVvOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        -1.0*np.einsum('Ae,BecIJK->ABcIJK', H.a.vv[Va, va], T.aab.VvvOOO, optimize=True)
        -1.0*np.einsum('AE,BEcIJK->ABcIJK', H.a.vv[Va, Va], T.aab.VVvOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        -1.0*np.einsum('ce,BAeIJK->ABcIJK', H.b.vv[vb, vb], T.aab.VVvOOO, optimize=True)
        -1.0*np.einsum('cE,BAEIJK->ABcIJK', H.b.vv[vb, Vb], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        -0.5*np.einsum('mnIJ,BAcmnK->ABcIJK', H.aa.oooo[oa, oa, Oa, Oa], T.aab.VVvooO, optimize=True)
        +1.0*np.einsum('MnIJ,BAcnMK->ABcIJK', H.aa.oooo[Oa, oa, Oa, Oa], T.aab.VVvoOO, optimize=True)
        -0.5*np.einsum('MNIJ,BAcMNK->ABcIJK', H.aa.oooo[Oa, Oa, Oa, Oa], T.aab.VVvOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('mnJK,BAcmIn->ABcIJK', H.ab.oooo[oa, ob, Oa, Ob], T.aab.VVvoOo, optimize=True)
        +1.0*np.einsum('mNJK,BAcmIN->ABcIJK', H.ab.oooo[oa, Ob, Oa, Ob], T.aab.VVvoOO, optimize=True)
        -1.0*np.einsum('MnJK,BAcIMn->ABcIJK', H.ab.oooo[Oa, ob, Oa, Ob], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('MNJK,BAcIMN->ABcIJK', H.ab.oooo[Oa, Ob, Oa, Ob], T.aab.VVvOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        +1.0*np.einsum('ABEf,EfcIJK->ABcIJK', H.aa.vvvv[Va, Va, Va, va], T.aab.VvvOOO, optimize=True)
        -0.5*np.einsum('ABEF,FEcIJK->ABcIJK', H.aa.vvvv[Va, Va, Va, Va], T.aab.VVvOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('Bcef,AefIJK->ABcIJK', H.ab.vvvv[Va, vb, va, vb], T.aab.VvvOOO, optimize=True)
        +1.0*np.einsum('BceF,AeFIJK->ABcIJK', H.ab.vvvv[Va, vb, va, Vb], T.aab.VvVOOO, optimize=True)
        -1.0*np.einsum('BcEf,EAfIJK->ABcIJK', H.ab.vvvv[Va, vb, Va, vb], T.aab.VVvOOO, optimize=True)
        -1.0*np.einsum('BcEF,EAFIJK->ABcIJK', H.ab.vvvv[Va, vb, Va, Vb], T.aab.VVVOOO, optimize=True)
)
x3b += (4.0/4.0) * (
        -1.0*np.einsum('AmIe,BecmJK->ABcIJK', H.aa.voov[Va, oa, Oa, va], T.aab.VvvoOO, optimize=True)
        -1.0*np.einsum('AMIe,BecMJK->ABcIJK', H.aa.voov[Va, Oa, Oa, va], T.aab.VvvOOO, optimize=True)
        -1.0*np.einsum('AmIE,BEcmJK->ABcIJK', H.aa.voov[Va, oa, Oa, Va], T.aab.VVvoOO, optimize=True)
        -1.0*np.einsum('AMIE,BEcMJK->ABcIJK', H.aa.voov[Va, Oa, Oa, Va], T.aab.VVvOOO, optimize=True)
)
x3b += (4.0/4.0) * (
        -1.0*np.einsum('AmIe,BceJmK->ABcIJK', H.ab.voov[Va, ob, Oa, vb], T.abb.VvvOoO, optimize=True)
        -1.0*np.einsum('AMIe,BceJMK->ABcIJK', H.ab.voov[Va, Ob, Oa, vb], T.abb.VvvOOO, optimize=True)
        +1.0*np.einsum('AmIE,BEcJmK->ABcIJK', H.ab.voov[Va, ob, Oa, Vb], T.abb.VVvOoO, optimize=True)
        +1.0*np.einsum('AMIE,BEcJMK->ABcIJK', H.ab.voov[Va, Ob, Oa, Vb], T.abb.VVvOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        -1.0*np.einsum('mceK,BAemIJ->ABcIJK', H.ab.ovvo[oa, vb, va, Ob], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('MceK,BAeIJM->ABcIJK', H.ab.ovvo[Oa, vb, va, Ob], T.aaa.VVvOOO, optimize=True)
        -1.0*np.einsum('mcEK,EBAmIJ->ABcIJK', H.ab.ovvo[oa, vb, Va, Ob], T.aaa.VVVoOO, optimize=True)
        -1.0*np.einsum('McEK,EBAIJM->ABcIJK', H.ab.ovvo[Oa, vb, Va, Ob], T.aaa.VVVOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        -1.0*np.einsum('cmKe,BAeIJm->ABcIJK', H.bb.voov[vb, ob, Ob, vb], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('cMKe,BAeIJM->ABcIJK', H.bb.voov[vb, Ob, Ob, vb], T.aab.VVvOOO, optimize=True)
        -1.0*np.einsum('cmKE,BAEIJm->ABcIJK', H.bb.voov[vb, ob, Ob, Vb], T.aab.VVVOOo, optimize=True)
        -1.0*np.einsum('cMKE,BAEIJM->ABcIJK', H.bb.voov[vb, Ob, Ob, Vb], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('AmeK,BecIJm->ABcIJK', H.ab.vovo[Va, ob, va, Ob], T.aab.VvvOOo, optimize=True)
        +1.0*np.einsum('AMeK,BecIJM->ABcIJK', H.ab.vovo[Va, Ob, va, Ob], T.aab.VvvOOO, optimize=True)
        +1.0*np.einsum('AmEK,BEcIJm->ABcIJK', H.ab.vovo[Va, ob, Va, Ob], T.aab.VVvOOo, optimize=True)
        +1.0*np.einsum('AMEK,BEcIJM->ABcIJK', H.ab.vovo[Va, Ob, Va, Ob], T.aab.VVvOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('mcIe,BAemJK->ABcIJK', H.ab.ovov[oa, vb, Oa, vb], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('McIe,BAeMJK->ABcIJK', H.ab.ovov[Oa, vb, Oa, vb], T.aab.VVvOOO, optimize=True)
        +1.0*np.einsum('mcIE,BAEmJK->ABcIJK', H.ab.ovov[oa, vb, Oa, Vb], T.aab.VVVoOO, optimize=True)
        +1.0*np.einsum('McIE,BAEMJK->ABcIJK', H.ab.ovov[Oa, vb, Oa, Vb], T.aab.VVVOOO, optimize=True)
)


x3b -= np.transpose(x3b, (1, 0, 2, 3, 4, 5))
x3b -= np.transpose(x3b, (0, 1, 2, 4, 3, 5))

get_error(X3B[Va, Va, vb, Oa, Oa, Ob], x3b)

1.752067186302736e-17


### Projection 3

In [13]:
x3b = (2.0/2.0) * (
        -1.0*np.einsum('mI,AbCmJK->AbCIJK', H.a.oo[oa, Oa], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('MI,AbCMJK->AbCIJK', H.a.oo[Oa, Oa], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('mK,AbCIJm->AbCIJK', H.b.oo[ob, Ob], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('MK,AbCIJM->AbCIJK', H.b.oo[Ob, Ob], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('Ae,beCIJK->AbCIJK', H.a.vv[Va, va], T.aab.vvVOOO, optimize=True)
        +1.0*np.einsum('AE,EbCIJK->AbCIJK', H.a.vv[Va, Va], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('be,AeCIJK->AbCIJK', H.a.vv[va, va], T.aab.VvVOOO, optimize=True)
        +1.0*np.einsum('bE,AECIJK->AbCIJK', H.a.vv[va, Va], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('Ce,AbeIJK->AbCIJK', H.b.vv[Vb, vb], T.aab.VvvOOO, optimize=True)
        +1.0*np.einsum('CE,AbEIJK->AbCIJK', H.b.vv[Vb, Vb], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +0.5*np.einsum('mnIJ,AbCmnK->AbCIJK', H.aa.oooo[oa, oa, Oa, Oa], T.aab.VvVooO, optimize=True)
        +1.0*np.einsum('mNIJ,AbCmNK->AbCIJK', H.aa.oooo[oa, Oa, Oa, Oa], T.aab.VvVoOO, optimize=True)
        +0.5*np.einsum('MNIJ,AbCMNK->AbCIJK', H.aa.oooo[Oa, Oa, Oa, Oa], T.aab.VvVOOO, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('mnJK,AbCmIn->AbCIJK', H.ab.oooo[oa, ob, Oa, Ob], T.aab.VvVoOo, optimize=True)
        +1.0*np.einsum('MnJK,AbCIMn->AbCIJK', H.ab.oooo[Oa, ob, Oa, Ob], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('mNJK,AbCmIN->AbCIJK', H.ab.oooo[oa, Ob, Oa, Ob], T.aab.VvVoOO, optimize=True)
        +1.0*np.einsum('MNJK,AbCIMN->AbCIJK', H.ab.oooo[Oa, Ob, Oa, Ob], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -0.5*np.einsum('Abef,feCIJK->AbCIJK', H.aa.vvvv[Va, va, va, va], T.aab.vvVOOO, optimize=True)
        +1.0*np.einsum('AbEf,EfCIJK->AbCIJK', H.aa.vvvv[Va, va, Va, va], T.aab.VvVOOO, optimize=True)
        -0.5*np.einsum('AbEF,FECIJK->AbCIJK', H.aa.vvvv[Va, va, Va, Va], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('bCef,AefIJK->AbCIJK', H.ab.vvvv[va, Vb, va, vb], T.aab.VvvOOO, optimize=True)
        +1.0*np.einsum('bCeF,AeFIJK->AbCIJK', H.ab.vvvv[va, Vb, va, Vb], T.aab.VvVOOO, optimize=True)
        -1.0*np.einsum('bCEf,EAfIJK->AbCIJK', H.ab.vvvv[va, Vb, Va, vb], T.aab.VVvOOO, optimize=True)
        -1.0*np.einsum('bCEF,EAFIJK->AbCIJK', H.ab.vvvv[va, Vb, Va, Vb], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('ACeF,ebFIJK->AbCIJK', H.ab.vvvv[Va, Vb, va, Vb], T.aab.vvVOOO, optimize=True)
        +1.0*np.einsum('ACEf,EbfIJK->AbCIJK', H.ab.vvvv[Va, Vb, Va, vb], T.aab.VvvOOO, optimize=True)
        +1.0*np.einsum('ACEF,EbFIJK->AbCIJK', H.ab.vvvv[Va, Vb, Va, Vb], T.aab.VvVOOO, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('AmIe,beCmJK->AbCIJK', H.aa.voov[Va, oa, Oa, va], T.aab.vvVoOO, optimize=True)
        -1.0*np.einsum('AMIe,beCMJK->AbCIJK', H.aa.voov[Va, Oa, Oa, va], T.aab.vvVOOO, optimize=True)
        +1.0*np.einsum('AmIE,EbCmJK->AbCIJK', H.aa.voov[Va, oa, Oa, Va], T.aab.VvVoOO, optimize=True)
        +1.0*np.einsum('AMIE,EbCMJK->AbCIJK', H.aa.voov[Va, Oa, Oa, Va], T.aab.VvVOOO, optimize=True)
)
x3b += (2.0/2.0) * (
        +1.0*np.einsum('bmIe,AeCmJK->AbCIJK', H.aa.voov[va, oa, Oa, va], T.aab.VvVoOO, optimize=True)
        +1.0*np.einsum('bMIe,AeCMJK->AbCIJK', H.aa.voov[va, Oa, Oa, va], T.aab.VvVOOO, optimize=True)
        +1.0*np.einsum('bmIE,AECmJK->AbCIJK', H.aa.voov[va, oa, Oa, Va], T.aab.VVVoOO, optimize=True)
        +1.0*np.einsum('bMIE,AECMJK->AbCIJK', H.aa.voov[va, Oa, Oa, Va], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('AmIe,bCeJmK->AbCIJK', H.ab.voov[Va, ob, Oa, vb], T.abb.vVvOoO, optimize=True)
        -1.0*np.einsum('AMIe,bCeJMK->AbCIJK', H.ab.voov[Va, Ob, Oa, vb], T.abb.vVvOOO, optimize=True)
        -1.0*np.einsum('AmIE,bCEJmK->AbCIJK', H.ab.voov[Va, ob, Oa, Vb], T.abb.vVVOoO, optimize=True)
        -1.0*np.einsum('AMIE,bCEJMK->AbCIJK', H.ab.voov[Va, Ob, Oa, Vb], T.abb.vVVOOO, optimize=True)
)
x3b += (2.0/2.0) * (
        +1.0*np.einsum('bmIe,ACeJmK->AbCIJK', H.ab.voov[va, ob, Oa, vb], T.abb.VVvOoO, optimize=True)
        +1.0*np.einsum('bMIe,ACeJMK->AbCIJK', H.ab.voov[va, Ob, Oa, vb], T.abb.VVvOOO, optimize=True)
        +1.0*np.einsum('bmIE,ACEJmK->AbCIJK', H.ab.voov[va, ob, Oa, Vb], T.abb.VVVOoO, optimize=True)
        +1.0*np.einsum('bMIE,ACEJMK->AbCIJK', H.ab.voov[va, Ob, Oa, Vb], T.abb.VVVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('mCeK,AebmIJ->AbCIJK', H.ab.ovvo[oa, Vb, va, Ob], T.aaa.VvvoOO, optimize=True)
        -1.0*np.einsum('MCeK,AebIJM->AbCIJK', H.ab.ovvo[Oa, Vb, va, Ob], T.aaa.VvvOOO, optimize=True)
        +1.0*np.einsum('mCEK,EAbmIJ->AbCIJK', H.ab.ovvo[oa, Vb, Va, Ob], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('MCEK,EAbIJM->AbCIJK', H.ab.ovvo[Oa, Vb, Va, Ob], T.aaa.VVvOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('CmKe,AbeIJm->AbCIJK', H.bb.voov[Vb, ob, Ob, vb], T.aab.VvvOOo, optimize=True)
        +1.0*np.einsum('CMKe,AbeIJM->AbCIJK', H.bb.voov[Vb, Ob, Ob, vb], T.aab.VvvOOO, optimize=True)
        +1.0*np.einsum('CmKE,AbEIJm->AbCIJK', H.bb.voov[Vb, ob, Ob, Vb], T.aab.VvVOOo, optimize=True)
        +1.0*np.einsum('CMKE,AbEIJM->AbCIJK', H.bb.voov[Vb, Ob, Ob, Vb], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('AmeK,beCIJm->AbCIJK', H.ab.vovo[Va, ob, va, Ob], T.aab.vvVOOo, optimize=True)
        +1.0*np.einsum('AMeK,beCIJM->AbCIJK', H.ab.vovo[Va, Ob, va, Ob], T.aab.vvVOOO, optimize=True)
        -1.0*np.einsum('AmEK,EbCIJm->AbCIJK', H.ab.vovo[Va, ob, Va, Ob], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('AMEK,EbCIJM->AbCIJK', H.ab.vovo[Va, Ob, Va, Ob], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('bmeK,AeCIJm->AbCIJK', H.ab.vovo[va, ob, va, Ob], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('bMeK,AeCIJM->AbCIJK', H.ab.vovo[va, Ob, va, Ob], T.aab.VvVOOO, optimize=True)
        -1.0*np.einsum('bmEK,AECIJm->AbCIJK', H.ab.vovo[va, ob, Va, Ob], T.aab.VVVOOo, optimize=True)
        -1.0*np.einsum('bMEK,AECIJM->AbCIJK', H.ab.vovo[va, Ob, Va, Ob], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('mCIe,AbemJK->AbCIJK', H.ab.ovov[oa, Vb, Oa, vb], T.aab.VvvoOO, optimize=True)
        -1.0*np.einsum('MCIe,AbeMJK->AbCIJK', H.ab.ovov[Oa, Vb, Oa, vb], T.aab.VvvOOO, optimize=True)
        -1.0*np.einsum('mCIE,AbEmJK->AbCIJK', H.ab.ovov[oa, Vb, Oa, Vb], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('MCIE,AbEMJK->AbCIJK', H.ab.ovov[Oa, Vb, Oa, Vb], T.aab.VvVOOO, optimize=True)
)

x3b -= np.transpose(x3b, (0, 1, 2, 4, 3, 5))

get_error(X3B[Va, va, Vb, Oa, Oa, Ob], x3b)

3.287679838687902e-17


### Projection 4

In [14]:
x3b = (2.0/4.0) * (
        +1.0*np.einsum('mI,BACmJk->ABCIJk', H.a.oo[oa, Oa], T.aab.VVVoOo, optimize=True)
        +1.0*np.einsum('MI,BACMJk->ABCIJk', H.a.oo[Oa, Oa], T.aab.VVVOOo, optimize=True)
)
x3b += (1.0/4.0) * (
        +1.0*np.einsum('mk,BACIJm->ABCIJk', H.b.oo[ob, ob], T.aab.VVVOOo, optimize=True)
        +1.0*np.einsum('Mk,BACIJM->ABCIJk', H.b.oo[Ob, ob], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        -1.0*np.einsum('Ae,BeCIJk->ABCIJk', H.a.vv[Va, va], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('AE,BECIJk->ABCIJk', H.a.vv[Va, Va], T.aab.VVVOOo, optimize=True)
)
x3b += (1.0/4.0) * (
        -1.0*np.einsum('Ce,BAeIJk->ABCIJk', H.b.vv[Vb, vb], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('CE,BAEIJk->ABCIJk', H.b.vv[Vb, Vb], T.aab.VVVOOo, optimize=True)
)
x3b += (1.0/4.0) * (
        +1.0*np.einsum('MnIJ,BACnMk->ABCIJk', H.aa.oooo[Oa, oa, Oa, Oa], T.aab.VVVoOo, optimize=True)
        -0.5*np.einsum('MNIJ,BACMNk->ABCIJk', H.aa.oooo[Oa, Oa, Oa, Oa], T.aab.VVVOOo, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('mnJk,BACmIn->ABCIJk', H.ab.oooo[oa, ob, Oa, ob], T.aab.VVVoOo, optimize=True)
        +1.0*np.einsum('mNJk,BACmIN->ABCIJk', H.ab.oooo[oa, Ob, Oa, ob], T.aab.VVVoOO, optimize=True)
        -1.0*np.einsum('MnJk,BACIMn->ABCIJk', H.ab.oooo[Oa, ob, Oa, ob], T.aab.VVVOOo, optimize=True)
        -1.0*np.einsum('MNJk,BACIMN->ABCIJk', H.ab.oooo[Oa, Ob, Oa, ob], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        -0.5*np.einsum('ABef,feCIJk->ABCIJk', H.aa.vvvv[Va, Va, va, va], T.aab.vvVOOo, optimize=True)
        -1.0*np.einsum('ABeF,FeCIJk->ABCIJk', H.aa.vvvv[Va, Va, va, Va], T.aab.VvVOOo, optimize=True)
        -0.5*np.einsum('ABEF,FECIJk->ABCIJk', H.aa.vvvv[Va, Va, Va, Va], T.aab.VVVOOo, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('BCef,AefIJk->ABCIJk', H.ab.vvvv[Va, Vb, va, vb], T.aab.VvvOOo, optimize=True)
        -1.0*np.einsum('BCEf,EAfIJk->ABCIJk', H.ab.vvvv[Va, Vb, Va, vb], T.aab.VVvOOo, optimize=True)
        +1.0*np.einsum('BCeF,AeFIJk->ABCIJk', H.ab.vvvv[Va, Vb, va, Vb], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('BCEF,EAFIJk->ABCIJk', H.ab.vvvv[Va, Vb, Va, Vb], T.aab.VVVOOo, optimize=True)
)
x3b += (4.0/4.0) * (
        -1.0*np.einsum('AmIe,BeCmJk->ABCIJk', H.aa.voov[Va, oa, Oa, va], T.aab.VvVoOo, optimize=True)
        -1.0*np.einsum('AmIE,BECmJk->ABCIJk', H.aa.voov[Va, oa, Oa, Va], T.aab.VVVoOo, optimize=True)
        -1.0*np.einsum('AMIe,BeCMJk->ABCIJk', H.aa.voov[Va, Oa, Oa, va], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('AMIE,BECMJk->ABCIJk', H.aa.voov[Va, Oa, Oa, Va], T.aab.VVVOOo, optimize=True)
)
x3b += (4.0/4.0) * (
        -1.0*np.einsum('AmIe,BCeJmk->ABCIJk', H.ab.voov[Va, ob, Oa, vb], T.abb.VVvOoo, optimize=True)
        -1.0*np.einsum('AmIE,BCEJmk->ABCIJk', H.ab.voov[Va, ob, Oa, Vb], T.abb.VVVOoo, optimize=True)
        +1.0*np.einsum('AMIe,BCeJkM->ABCIJk', H.ab.voov[Va, Ob, Oa, vb], T.abb.VVvOoO, optimize=True)
        +1.0*np.einsum('AMIE,BCEJkM->ABCIJk', H.ab.voov[Va, Ob, Oa, Vb], T.abb.VVVOoO, optimize=True)
)
x3b += (1.0/4.0) * (
        -1.0*np.einsum('mCek,BAemIJ->ABCIJk', H.ab.ovvo[oa, Vb, va, ob], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('mCEk,EBAmIJ->ABCIJk', H.ab.ovvo[oa, Vb, Va, ob], T.aaa.VVVoOO, optimize=True)
        -1.0*np.einsum('MCek,BAeIJM->ABCIJk', H.ab.ovvo[Oa, Vb, va, ob], T.aaa.VVvOOO, optimize=True)
        -1.0*np.einsum('MCEk,EBAIJM->ABCIJk', H.ab.ovvo[Oa, Vb, Va, ob], T.aaa.VVVOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        -1.0*np.einsum('Cmke,BAeIJm->ABCIJk', H.bb.voov[Vb, ob, ob, vb], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('CmkE,BAEIJm->ABCIJk', H.bb.voov[Vb, ob, ob, Vb], T.aab.VVVOOo, optimize=True)
        -1.0*np.einsum('CMke,BAeIJM->ABCIJk', H.bb.voov[Vb, Ob, ob, vb], T.aab.VVvOOO, optimize=True)
        -1.0*np.einsum('CMkE,BAEIJM->ABCIJk', H.bb.voov[Vb, Ob, ob, Vb], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('Amek,BeCIJm->ABCIJk', H.ab.vovo[Va, ob, va, ob], T.aab.VvVOOo, optimize=True)
        +1.0*np.einsum('AmEk,BECIJm->ABCIJk', H.ab.vovo[Va, ob, Va, ob], T.aab.VVVOOo, optimize=True)
        +1.0*np.einsum('AMek,BeCIJM->ABCIJk', H.ab.vovo[Va, Ob, va, ob], T.aab.VvVOOO, optimize=True)
        +1.0*np.einsum('AMEk,BECIJM->ABCIJk', H.ab.vovo[Va, Ob, Va, ob], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('mCIe,BAemJk->ABCIJk', H.ab.ovov[oa, Vb, Oa, vb], T.aab.VVvoOo, optimize=True)
        +1.0*np.einsum('mCIE,BAEmJk->ABCIJk', H.ab.ovov[oa, Vb, Oa, Vb], T.aab.VVVoOo, optimize=True)
        +1.0*np.einsum('MCIe,BAeMJk->ABCIJk', H.ab.ovov[Oa, Vb, Oa, vb], T.aab.VVvOOo, optimize=True)
        +1.0*np.einsum('MCIE,BAEMJk->ABCIJk', H.ab.ovov[Oa, Vb, Oa, Vb], T.aab.VVVOOo, optimize=True)
)

x3b -= np.transpose(x3b, (1, 0, 2, 3, 4, 5))
x3b -= np.transpose(x3b, (0, 1, 2, 4, 3, 5))

get_error(X3B[Va, Va, Vb, Oa, Oa, ob], x3b)

1.031053419380902e-17


### Projection 5

In [15]:
x3b = (1.0/2.0) * (
        +1.0*np.einsum('mi,BACmJK->ABCiJK', H.a.oo[oa, oa], T.aab.VVVoOO, optimize=True)
        +1.0*np.einsum('Mi,BACMJK->ABCiJK', H.a.oo[Oa, oa], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('mJ,BACmiK->ABCiJK', H.a.oo[oa, Oa], T.aab.VVVooO, optimize=True)
        +1.0*np.einsum('MJ,BACiMK->ABCiJK', H.a.oo[Oa, Oa], T.aab.VVVoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('mK,BACiJm->ABCiJK', H.b.oo[ob, Ob], T.aab.VVVoOo, optimize=True)
        +1.0*np.einsum('MK,BACiJM->ABCiJK', H.b.oo[Ob, Ob], T.aab.VVVoOO, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('Ae,BeCiJK->ABCiJK', H.a.vv[Va, va], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('AE,BECiJK->ABCiJK', H.a.vv[Va, Va], T.aab.VVVoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('Ce,BAeiJK->ABCiJK', H.b.vv[Vb, vb], T.aab.VVvoOO, optimize=True)
        -1.0*np.einsum('CE,BAEiJK->ABCiJK', H.b.vv[Vb, Vb], T.aab.VVVoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -0.5*np.einsum('mniJ,BACmnK->ABCiJK', H.aa.oooo[oa, oa, oa, Oa], T.aab.VVVooO, optimize=True)
        +1.0*np.einsum('MniJ,BACnMK->ABCiJK', H.aa.oooo[Oa, oa, oa, Oa], T.aab.VVVoOO, optimize=True)
        -0.5*np.einsum('MNiJ,BACMNK->ABCiJK', H.aa.oooo[Oa, Oa, oa, Oa], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('mNJK,BACimN->ABCiJK', H.ab.oooo[oa, Ob, Oa, Ob], T.aab.VVVooO, optimize=True)
        -1.0*np.einsum('MnJK,BACiMn->ABCiJK', H.ab.oooo[Oa, ob, Oa, Ob], T.aab.VVVoOo, optimize=True)
        -1.0*np.einsum('MNJK,BACiMN->ABCiJK', H.ab.oooo[Oa, Ob, Oa, Ob], T.aab.VVVoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('mniK,BACmJn->ABCiJK', H.ab.oooo[oa, ob, oa, Ob], T.aab.VVVoOo, optimize=True)
        -1.0*np.einsum('mNiK,BACmJN->ABCiJK', H.ab.oooo[oa, Ob, oa, Ob], T.aab.VVVoOO, optimize=True)
        +1.0*np.einsum('MniK,BACJMn->ABCiJK', H.ab.oooo[Oa, ob, oa, Ob], T.aab.VVVOOo, optimize=True)
        +1.0*np.einsum('MNiK,BACJMN->ABCiJK', H.ab.oooo[Oa, Ob, oa, Ob], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -0.5*np.einsum('ABef,feCiJK->ABCiJK', H.aa.vvvv[Va, Va, va, va], T.aab.vvVoOO, optimize=True)
        -1.0*np.einsum('ABeF,FeCiJK->ABCiJK', H.aa.vvvv[Va, Va, va, Va], T.aab.VvVoOO, optimize=True)
        -0.5*np.einsum('ABEF,FECiJK->ABCiJK', H.aa.vvvv[Va, Va, Va, Va], T.aab.VVVoOO, optimize=True)
)
x3b += (2.0/2.0) * (
        +1.0*np.einsum('BCef,AefiJK->ABCiJK', H.ab.vvvv[Va, Vb, va, vb], T.aab.VvvoOO, optimize=True)
        -1.0*np.einsum('BCEf,EAfiJK->ABCiJK', H.ab.vvvv[Va, Vb, Va, vb], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('BCeF,AeFiJK->ABCiJK', H.ab.vvvv[Va, Vb, va, Vb], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('BCEF,EAFiJK->ABCiJK', H.ab.vvvv[Va, Vb, Va, Vb], T.aab.VVVoOO, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('Amie,BeCmJK->ABCiJK', H.aa.voov[Va, oa, oa, va], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('AmiE,BECmJK->ABCiJK', H.aa.voov[Va, oa, oa, Va], T.aab.VVVoOO, optimize=True)
        -1.0*np.einsum('AMie,BeCMJK->ABCiJK', H.aa.voov[Va, Oa, oa, va], T.aab.VvVOOO, optimize=True)
        -1.0*np.einsum('AMiE,BECMJK->ABCiJK', H.aa.voov[Va, Oa, oa, Va], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/2.0) * (
        +1.0*np.einsum('AmJe,BeCmiK->ABCiJK', H.aa.voov[Va, oa, Oa, va], T.aab.VvVooO, optimize=True)
        +1.0*np.einsum('AmJE,BECmiK->ABCiJK', H.aa.voov[Va, oa, Oa, Va], T.aab.VVVooO, optimize=True)
        -1.0*np.einsum('AMJe,BeCiMK->ABCiJK', H.aa.voov[Va, Oa, Oa, va], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('AMJE,BECiMK->ABCiJK', H.aa.voov[Va, Oa, Oa, Va], T.aab.VVVoOO, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('Amie,BCeJmK->ABCiJK', H.ab.voov[Va, ob, oa, vb], T.abb.VVvOoO, optimize=True)
        -1.0*np.einsum('AmiE,BCEJmK->ABCiJK', H.ab.voov[Va, ob, oa, Vb], T.abb.VVVOoO, optimize=True)
        -1.0*np.einsum('AMie,BCeJMK->ABCiJK', H.ab.voov[Va, Ob, oa, vb], T.abb.VVvOOO, optimize=True)
        -1.0*np.einsum('AMiE,BCEJMK->ABCiJK', H.ab.voov[Va, Ob, oa, Vb], T.abb.VVVOOO, optimize=True)
)
x3b += (2.0/2.0) * (
        +1.0*np.einsum('AmJe,BCeimK->ABCiJK', H.ab.voov[Va, ob, Oa, vb], T.abb.VVvooO, optimize=True)
        +1.0*np.einsum('AmJE,BCEimK->ABCiJK', H.ab.voov[Va, ob, Oa, Vb], T.abb.VVVooO, optimize=True)
        +1.0*np.einsum('AMJe,BCeiMK->ABCiJK', H.ab.voov[Va, Ob, Oa, vb], T.abb.VVvoOO, optimize=True)
        +1.0*np.einsum('AMJE,BCEiMK->ABCiJK', H.ab.voov[Va, Ob, Oa, Vb], T.abb.VVVoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('mCeK,BAeimJ->ABCiJK', H.ab.ovvo[oa, Vb, va, Ob], T.aaa.VVvooO, optimize=True)
        +1.0*np.einsum('mCEK,EBAimJ->ABCiJK', H.ab.ovvo[oa, Vb, Va, Ob], T.aaa.VVVooO, optimize=True)
        -1.0*np.einsum('MCeK,BAeiJM->ABCiJK', H.ab.ovvo[Oa, Vb, va, Ob], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('MCEK,EBAiJM->ABCiJK', H.ab.ovvo[Oa, Vb, Va, Ob], T.aaa.VVVoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('CmKe,BAeiJm->ABCiJK', H.bb.voov[Vb, ob, Ob, vb], T.aab.VVvoOo, optimize=True)
        -1.0*np.einsum('CmKE,BAEiJm->ABCiJK', H.bb.voov[Vb, ob, Ob, Vb], T.aab.VVVoOo, optimize=True)
        -1.0*np.einsum('CMKe,BAeiJM->ABCiJK', H.bb.voov[Vb, Ob, Ob, vb], T.aab.VVvoOO, optimize=True)
        -1.0*np.einsum('CMKE,BAEiJM->ABCiJK', H.bb.voov[Vb, Ob, Ob, Vb], T.aab.VVVoOO, optimize=True)
)
x3b += (2.0/2.0) * (
        +1.0*np.einsum('AmeK,BeCiJm->ABCiJK', H.ab.vovo[Va, ob, va, Ob], T.aab.VvVoOo, optimize=True)
        +1.0*np.einsum('AmEK,BECiJm->ABCiJK', H.ab.vovo[Va, ob, Va, Ob], T.aab.VVVoOo, optimize=True)
        +1.0*np.einsum('AMeK,BeCiJM->ABCiJK', H.ab.vovo[Va, Ob, va, Ob], T.aab.VvVoOO, optimize=True)
        +1.0*np.einsum('AMEK,BECiJM->ABCiJK', H.ab.vovo[Va, Ob, Va, Ob], T.aab.VVVoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('mCie,BAemJK->ABCiJK', H.ab.ovov[oa, Vb, oa, vb], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('mCiE,BAEmJK->ABCiJK', H.ab.ovov[oa, Vb, oa, Vb], T.aab.VVVoOO, optimize=True)
        +1.0*np.einsum('MCie,BAeMJK->ABCiJK', H.ab.ovov[Oa, Vb, oa, vb], T.aab.VVvOOO, optimize=True)
        +1.0*np.einsum('MCiE,BAEMJK->ABCiJK', H.ab.ovov[Oa, Vb, oa, Vb], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('mCJe,BAemiK->ABCiJK', H.ab.ovov[oa, Vb, Oa, vb], T.aab.VVvooO, optimize=True)
        -1.0*np.einsum('mCJE,BAEmiK->ABCiJK', H.ab.ovov[oa, Vb, Oa, Vb], T.aab.VVVooO, optimize=True)
        +1.0*np.einsum('MCJe,BAeiMK->ABCiJK', H.ab.ovov[Oa, Vb, Oa, vb], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('MCJE,BAEiMK->ABCiJK', H.ab.ovov[Oa, Vb, Oa, Vb], T.aab.VVVoOO, optimize=True)
)


x3b -= np.transpose(x3b, (1, 0, 2, 3, 4, 5))

get_error(X3B[Va, Va, Vb, oa, Oa, Ob], x3b)

6.174070106904578e-17


### Projection 6

In [16]:
x3b = (1.0/1.0) * (
        -1.0*np.einsum('mi,AbCmJK->AbCiJK', H.a.oo[oa, oa], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('Mi,AbCMJK->AbCiJK', H.a.oo[Oa, oa], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('mJ,AbCmiK->AbCiJK', H.a.oo[oa, Oa], T.aab.VvVooO, optimize=True)
        -1.0*np.einsum('MJ,AbCiMK->AbCiJK', H.a.oo[Oa, Oa], T.aab.VvVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        -1.0*np.einsum('mK,AbCiJm->AbCiJK', H.b.oo[ob, Ob], T.aab.VvVoOo, optimize=True)
        -1.0*np.einsum('MK,AbCiJM->AbCiJK', H.b.oo[Ob, Ob], T.aab.VvVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        -1.0*np.einsum('Ae,beCiJK->AbCiJK', H.a.vv[Va, va], T.aab.vvVoOO, optimize=True)
        +1.0*np.einsum('AE,EbCiJK->AbCiJK', H.a.vv[Va, Va], T.aab.VvVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('be,AeCiJK->AbCiJK', H.a.vv[va, va], T.aab.VvVoOO, optimize=True)
        +1.0*np.einsum('bE,AECiJK->AbCiJK', H.a.vv[va, Va], T.aab.VVVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('Ce,AbeiJK->AbCiJK', H.b.vv[Vb, vb], T.aab.VvvoOO, optimize=True)
        +1.0*np.einsum('CE,AbEiJK->AbCiJK', H.b.vv[Vb, Vb], T.aab.VvVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +0.5*np.einsum('mniJ,AbCmnK->AbCiJK', H.aa.oooo[oa, oa, oa, Oa], T.aab.VvVooO, optimize=True)
        +1.0*np.einsum('mNiJ,AbCmNK->AbCiJK', H.aa.oooo[oa, Oa, oa, Oa], T.aab.VvVoOO, optimize=True)
        +0.5*np.einsum('MNiJ,AbCMNK->AbCiJK', H.aa.oooo[Oa, Oa, oa, Oa], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('MnJK,AbCiMn->AbCiJK', H.ab.oooo[Oa, ob, Oa, Ob], T.aab.VvVoOo, optimize=True)
        +1.0*np.einsum('mNJK,AbCimN->AbCiJK', H.ab.oooo[oa, Ob, Oa, Ob], T.aab.VvVooO, optimize=True)
        +1.0*np.einsum('MNJK,AbCiMN->AbCiJK', H.ab.oooo[Oa, Ob, Oa, Ob], T.aab.VvVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('mniK,AbCmJn->AbCiJK', H.ab.oooo[oa, ob, oa, Ob], T.aab.VvVoOo, optimize=True)
        -1.0*np.einsum('MniK,AbCJMn->AbCiJK', H.ab.oooo[Oa, ob, oa, Ob], T.aab.VvVOOo, optimize=True)
        +1.0*np.einsum('mNiK,AbCmJN->AbCiJK', H.ab.oooo[oa, Ob, oa, Ob], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('MNiK,AbCJMN->AbCiJK', H.ab.oooo[Oa, Ob, oa, Ob], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/1.0) * (
        -0.5*np.einsum('Abef,feCiJK->AbCiJK', H.aa.vvvv[Va, va, va, va], T.aab.vvVoOO, optimize=True)
        -1.0*np.einsum('AbeF,FeCiJK->AbCiJK', H.aa.vvvv[Va, va, va, Va], T.aab.VvVoOO, optimize=True)
        -0.5*np.einsum('AbEF,FECiJK->AbCiJK', H.aa.vvvv[Va, va, Va, Va], T.aab.VVVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('bCef,AefiJK->AbCiJK', H.ab.vvvv[va, Vb, va, vb], T.aab.VvvoOO, optimize=True)
        -1.0*np.einsum('bCEf,EAfiJK->AbCiJK', H.ab.vvvv[va, Vb, Va, vb], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('bCeF,AeFiJK->AbCiJK', H.ab.vvvv[va, Vb, va, Vb], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('bCEF,EAFiJK->AbCiJK', H.ab.vvvv[va, Vb, Va, Vb], T.aab.VVVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('ACEf,EbfiJK->AbCiJK', H.ab.vvvv[Va, Vb, Va, vb], T.aab.VvvoOO, optimize=True)
        +1.0*np.einsum('ACeF,ebFiJK->AbCiJK', H.ab.vvvv[Va, Vb, va, Vb], T.aab.vvVoOO, optimize=True)
        +1.0*np.einsum('ACEF,EbFiJK->AbCiJK', H.ab.vvvv[Va, Vb, Va, Vb], T.aab.VvVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        -1.0*np.einsum('Amie,beCmJK->AbCiJK', H.aa.voov[Va, oa, oa, va], T.aab.vvVoOO, optimize=True)
        -1.0*np.einsum('AMie,beCMJK->AbCiJK', H.aa.voov[Va, Oa, oa, va], T.aab.vvVOOO, optimize=True)
        +1.0*np.einsum('AmiE,EbCmJK->AbCiJK', H.aa.voov[Va, oa, oa, Va], T.aab.VvVoOO, optimize=True)
        +1.0*np.einsum('AMiE,EbCMJK->AbCiJK', H.aa.voov[Va, Oa, oa, Va], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('bmie,AeCmJK->AbCiJK', H.aa.voov[va, oa, oa, va], T.aab.VvVoOO, optimize=True)
        +1.0*np.einsum('bMie,AeCMJK->AbCiJK', H.aa.voov[va, Oa, oa, va], T.aab.VvVOOO, optimize=True)
        +1.0*np.einsum('bmiE,AECmJK->AbCiJK', H.aa.voov[va, oa, oa, Va], T.aab.VVVoOO, optimize=True)
        +1.0*np.einsum('bMiE,AECMJK->AbCiJK', H.aa.voov[va, Oa, oa, Va], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('AmJe,beCmiK->AbCiJK', H.aa.voov[Va, oa, Oa, va], T.aab.vvVooO, optimize=True)
        -1.0*np.einsum('AMJe,beCiMK->AbCiJK', H.aa.voov[Va, Oa, Oa, va], T.aab.vvVoOO, optimize=True)
        -1.0*np.einsum('AmJE,EbCmiK->AbCiJK', H.aa.voov[Va, oa, Oa, Va], T.aab.VvVooO, optimize=True)
        +1.0*np.einsum('AMJE,EbCiMK->AbCiJK', H.aa.voov[Va, Oa, Oa, Va], T.aab.VvVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        -1.0*np.einsum('bmJe,AeCmiK->AbCiJK', H.aa.voov[va, oa, Oa, va], T.aab.VvVooO, optimize=True)
        +1.0*np.einsum('bMJe,AeCiMK->AbCiJK', H.aa.voov[va, Oa, Oa, va], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('bmJE,AECmiK->AbCiJK', H.aa.voov[va, oa, Oa, Va], T.aab.VVVooO, optimize=True)
        +1.0*np.einsum('bMJE,AECiMK->AbCiJK', H.aa.voov[va, Oa, Oa, Va], T.aab.VVVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        -1.0*np.einsum('Amie,bCeJmK->AbCiJK', H.ab.voov[Va, ob, oa, vb], T.abb.vVvOoO, optimize=True)
        -1.0*np.einsum('AMie,bCeJMK->AbCiJK', H.ab.voov[Va, Ob, oa, vb], T.abb.vVvOOO, optimize=True)
        -1.0*np.einsum('AmiE,bCEJmK->AbCiJK', H.ab.voov[Va, ob, oa, Vb], T.abb.vVVOoO, optimize=True)
        -1.0*np.einsum('AMiE,bCEJMK->AbCiJK', H.ab.voov[Va, Ob, oa, Vb], T.abb.vVVOOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('bmie,ACeJmK->AbCiJK', H.ab.voov[va, ob, oa, vb], T.abb.VVvOoO, optimize=True)
        +1.0*np.einsum('bMie,ACeJMK->AbCiJK', H.ab.voov[va, Ob, oa, vb], T.abb.VVvOOO, optimize=True)
        +1.0*np.einsum('bmiE,ACEJmK->AbCiJK', H.ab.voov[va, ob, oa, Vb], T.abb.VVVOoO, optimize=True)
        +1.0*np.einsum('bMiE,ACEJMK->AbCiJK', H.ab.voov[va, Ob, oa, Vb], T.abb.VVVOOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('AmJe,bCeimK->AbCiJK', H.ab.voov[Va, ob, Oa, vb], T.abb.vVvooO, optimize=True)
        +1.0*np.einsum('AMJe,bCeiMK->AbCiJK', H.ab.voov[Va, Ob, Oa, vb], T.abb.vVvoOO, optimize=True)
        +1.0*np.einsum('AmJE,bCEimK->AbCiJK', H.ab.voov[Va, ob, Oa, Vb], T.abb.vVVooO, optimize=True)
        +1.0*np.einsum('AMJE,bCEiMK->AbCiJK', H.ab.voov[Va, Ob, Oa, Vb], T.abb.vVVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        -1.0*np.einsum('bmJe,ACeimK->AbCiJK', H.ab.voov[va, ob, Oa, vb], T.abb.VVvooO, optimize=True)
        -1.0*np.einsum('bMJe,ACeiMK->AbCiJK', H.ab.voov[va, Ob, Oa, vb], T.abb.VVvoOO, optimize=True)
        -1.0*np.einsum('bmJE,ACEimK->AbCiJK', H.ab.voov[va, ob, Oa, Vb], T.abb.VVVooO, optimize=True)
        -1.0*np.einsum('bMJE,ACEiMK->AbCiJK', H.ab.voov[va, Ob, Oa, Vb], T.abb.VVVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('mCeK,AebimJ->AbCiJK', H.ab.ovvo[oa, Vb, va, Ob], T.aaa.VvvooO, optimize=True)
        -1.0*np.einsum('MCeK,AebiJM->AbCiJK', H.ab.ovvo[Oa, Vb, va, Ob], T.aaa.VvvoOO, optimize=True)
        -1.0*np.einsum('mCEK,EAbimJ->AbCiJK', H.ab.ovvo[oa, Vb, Va, Ob], T.aaa.VVvooO, optimize=True)
        +1.0*np.einsum('MCEK,EAbiJM->AbCiJK', H.ab.ovvo[Oa, Vb, Va, Ob], T.aaa.VVvoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('CmKe,AbeiJm->AbCiJK', H.bb.voov[Vb, ob, Ob, vb], T.aab.VvvoOo, optimize=True)
        +1.0*np.einsum('CMKe,AbeiJM->AbCiJK', H.bb.voov[Vb, Ob, Ob, vb], T.aab.VvvoOO, optimize=True)
        +1.0*np.einsum('CmKE,AbEiJm->AbCiJK', H.bb.voov[Vb, ob, Ob, Vb], T.aab.VvVoOo, optimize=True)
        +1.0*np.einsum('CMKE,AbEiJM->AbCiJK', H.bb.voov[Vb, Ob, Ob, Vb], T.aab.VvVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('AmeK,beCiJm->AbCiJK', H.ab.vovo[Va, ob, va, Ob], T.aab.vvVoOo, optimize=True)
        +1.0*np.einsum('AMeK,beCiJM->AbCiJK', H.ab.vovo[Va, Ob, va, Ob], T.aab.vvVoOO, optimize=True)
        -1.0*np.einsum('AmEK,EbCiJm->AbCiJK', H.ab.vovo[Va, ob, Va, Ob], T.aab.VvVoOo, optimize=True)
        -1.0*np.einsum('AMEK,EbCiJM->AbCiJK', H.ab.vovo[Va, Ob, Va, Ob], T.aab.VvVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        -1.0*np.einsum('bmeK,AeCiJm->AbCiJK', H.ab.vovo[va, ob, va, Ob], T.aab.VvVoOo, optimize=True)
        -1.0*np.einsum('bMeK,AeCiJM->AbCiJK', H.ab.vovo[va, Ob, va, Ob], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('bmEK,AECiJm->AbCiJK', H.ab.vovo[va, ob, Va, Ob], T.aab.VVVoOo, optimize=True)
        -1.0*np.einsum('bMEK,AECiJM->AbCiJK', H.ab.vovo[va, Ob, Va, Ob], T.aab.VVVoOO, optimize=True)
)
x3b += (1.0/1.0) * (
        -1.0*np.einsum('mCie,AbemJK->AbCiJK', H.ab.ovov[oa, Vb, oa, vb], T.aab.VvvoOO, optimize=True)
        -1.0*np.einsum('MCie,AbeMJK->AbCiJK', H.ab.ovov[Oa, Vb, oa, vb], T.aab.VvvOOO, optimize=True)
        -1.0*np.einsum('mCiE,AbEmJK->AbCiJK', H.ab.ovov[oa, Vb, oa, Vb], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('MCiE,AbEMJK->AbCiJK', H.ab.ovov[Oa, Vb, oa, Vb], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/1.0) * (
        +1.0*np.einsum('mCJe,AbemiK->AbCiJK', H.ab.ovov[oa, Vb, Oa, vb], T.aab.VvvooO, optimize=True)
        -1.0*np.einsum('MCJe,AbeiMK->AbCiJK', H.ab.ovov[Oa, Vb, Oa, vb], T.aab.VvvoOO, optimize=True)
        +1.0*np.einsum('mCJE,AbEmiK->AbCiJK', H.ab.ovov[oa, Vb, Oa, Vb], T.aab.VvVooO, optimize=True)
        -1.0*np.einsum('MCJE,AbEiMK->AbCiJK', H.ab.ovov[Oa, Vb, Oa, Vb], T.aab.VvVoOO, optimize=True)
)


get_error(X3B[Va, va, Vb, oa, Oa, Ob], x3b)

6.114474506434402e-17


### Projection 7

In [17]:
x3b = (2.0/4.0) * (
        +1.0*np.einsum('mI,BAcmJk->ABcIJk', H.a.oo[oa, Oa], T.aab.VVvoOo, optimize=True)
        +1.0*np.einsum('MI,BAcMJk->ABcIJk', H.a.oo[Oa, Oa], T.aab.VVvOOo, optimize=True)
)
x3b += (1.0/4.0) * (
        +1.0*np.einsum('mk,BAcIJm->ABcIJk', H.b.oo[ob, ob], T.aab.VVvOOo, optimize=True)
        +1.0*np.einsum('Mk,BAcIJM->ABcIJk', H.b.oo[Ob, ob], T.aab.VVvOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        -1.0*np.einsum('Ae,BecIJk->ABcIJk', H.a.vv[Va, va], T.aab.VvvOOo, optimize=True)
        -1.0*np.einsum('AE,BEcIJk->ABcIJk', H.a.vv[Va, Va], T.aab.VVvOOo, optimize=True)
)
x3b += (1.0/4.0) * (
        -1.0*np.einsum('ce,BAeIJk->ABcIJk', H.b.vv[vb, vb], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('cE,BAEIJk->ABcIJk', H.b.vv[vb, Vb], T.aab.VVVOOo, optimize=True)
)
x3b += (1.0/4.0) * (
        +1.0*np.einsum('MnIJ,BAcnMk->ABcIJk', H.aa.oooo[Oa, oa, Oa, Oa], T.aab.VVvoOo, optimize=True)
        -0.5*np.einsum('MNIJ,BAcMNk->ABcIJk', H.aa.oooo[Oa, Oa, Oa, Oa], T.aab.VVvOOo, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('mnJk,BAcmIn->ABcIJk', H.ab.oooo[oa, ob, Oa, ob], T.aab.VVvoOo, optimize=True)
        +1.0*np.einsum('mNJk,BAcmIN->ABcIJk', H.ab.oooo[oa, Ob, Oa, ob], T.aab.VVvoOO, optimize=True)
        -1.0*np.einsum('MnJk,BAcIMn->ABcIJk', H.ab.oooo[Oa, ob, Oa, ob], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('MNJk,BAcIMN->ABcIJk', H.ab.oooo[Oa, Ob, Oa, ob], T.aab.VVvOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        +1.0*np.einsum('ABEf,EfcIJk->ABcIJk', H.aa.vvvv[Va, Va, Va, va], T.aab.VvvOOo, optimize=True)
        -0.5*np.einsum('ABEF,FEcIJk->ABcIJk', H.aa.vvvv[Va, Va, Va, Va], T.aab.VVvOOo, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('Bcef,AefIJk->ABcIJk', H.ab.vvvv[Va, vb, va, vb], T.aab.VvvOOo, optimize=True)
        +1.0*np.einsum('BceF,AeFIJk->ABcIJk', H.ab.vvvv[Va, vb, va, Vb], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('BcEf,EAfIJk->ABcIJk', H.ab.vvvv[Va, vb, Va, vb], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('BcEF,EAFIJk->ABcIJk', H.ab.vvvv[Va, vb, Va, Vb], T.aab.VVVOOo, optimize=True)
)
x3b += (4.0/4.0) * (
        -1.0*np.einsum('AmIe,BecmJk->ABcIJk', H.aa.voov[Va, oa, Oa, va], T.aab.VvvoOo, optimize=True)
        -1.0*np.einsum('AmIE,BEcmJk->ABcIJk', H.aa.voov[Va, oa, Oa, Va], T.aab.VVvoOo, optimize=True)
        -1.0*np.einsum('AMIe,BecMJk->ABcIJk', H.aa.voov[Va, Oa, Oa, va], T.aab.VvvOOo, optimize=True)
        -1.0*np.einsum('AMIE,BEcMJk->ABcIJk', H.aa.voov[Va, Oa, Oa, Va], T.aab.VVvOOo, optimize=True)
)
x3b += (4.0/4.0) * (
        -1.0*np.einsum('AmIe,BceJmk->ABcIJk', H.ab.voov[Va, ob, Oa, vb], T.abb.VvvOoo, optimize=True)
        +1.0*np.einsum('AmIE,BEcJmk->ABcIJk', H.ab.voov[Va, ob, Oa, Vb], T.abb.VVvOoo, optimize=True)
        +1.0*np.einsum('AMIe,BceJkM->ABcIJk', H.ab.voov[Va, Ob, Oa, vb], T.abb.VvvOoO, optimize=True)
        -1.0*np.einsum('AMIE,BEcJkM->ABcIJk', H.ab.voov[Va, Ob, Oa, Vb], T.abb.VVvOoO, optimize=True)
)
x3b += (1.0/4.0) * (
        -1.0*np.einsum('mcek,BAemIJ->ABcIJk', H.ab.ovvo[oa, vb, va, ob], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('mcEk,EBAmIJ->ABcIJk', H.ab.ovvo[oa, vb, Va, ob], T.aaa.VVVoOO, optimize=True)
        -1.0*np.einsum('Mcek,BAeIJM->ABcIJk', H.ab.ovvo[Oa, vb, va, ob], T.aaa.VVvOOO, optimize=True)
        -1.0*np.einsum('McEk,EBAIJM->ABcIJk', H.ab.ovvo[Oa, vb, Va, ob], T.aaa.VVVOOO, optimize=True)
)
x3b += (1.0/4.0) * (
        -1.0*np.einsum('cmke,BAeIJm->ABcIJk', H.bb.voov[vb, ob, ob, vb], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('cmkE,BAEIJm->ABcIJk', H.bb.voov[vb, ob, ob, Vb], T.aab.VVVOOo, optimize=True)
        -1.0*np.einsum('cMke,BAeIJM->ABcIJk', H.bb.voov[vb, Ob, ob, vb], T.aab.VVvOOO, optimize=True)
        -1.0*np.einsum('cMkE,BAEIJM->ABcIJk', H.bb.voov[vb, Ob, ob, Vb], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('Amek,BecIJm->ABcIJk', H.ab.vovo[Va, ob, va, ob], T.aab.VvvOOo, optimize=True)
        +1.0*np.einsum('AmEk,BEcIJm->ABcIJk', H.ab.vovo[Va, ob, Va, ob], T.aab.VVvOOo, optimize=True)
        +1.0*np.einsum('AMek,BecIJM->ABcIJk', H.ab.vovo[Va, Ob, va, ob], T.aab.VvvOOO, optimize=True)
        +1.0*np.einsum('AMEk,BEcIJM->ABcIJk', H.ab.vovo[Va, Ob, Va, ob], T.aab.VVvOOO, optimize=True)
)
x3b += (2.0/4.0) * (
        +1.0*np.einsum('mcIe,BAemJk->ABcIJk', H.ab.ovov[oa, vb, Oa, vb], T.aab.VVvoOo, optimize=True)
        +1.0*np.einsum('mcIE,BAEmJk->ABcIJk', H.ab.ovov[oa, vb, Oa, Vb], T.aab.VVVoOo, optimize=True)
        +1.0*np.einsum('McIe,BAeMJk->ABcIJk', H.ab.ovov[Oa, vb, Oa, vb], T.aab.VVvOOo, optimize=True)
        +1.0*np.einsum('McIE,BAEMJk->ABcIJk', H.ab.ovov[Oa, vb, Oa, Vb], T.aab.VVVOOo, optimize=True)
)

x3b -= np.transpose(x3b, (1, 0, 2, 3, 4, 5))
x3b -= np.transpose(x3b, (0, 1, 2, 4, 3, 5))

get_error(X3B[Va, Va, vb, Oa, Oa, ob], x3b)

3.6223343765888e-18


### Projection 8

In [18]:
x3b = (1.0/2.0) * (
        +1.0*np.einsum('mi,BAcmJK->ABciJK', H.a.oo[oa, oa], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('Mi,BAcMJK->ABciJK', H.a.oo[Oa, oa], T.aab.VVvOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('mJ,BAcmiK->ABciJK', H.a.oo[oa, Oa], T.aab.VVvooO, optimize=True)
        +1.0*np.einsum('MJ,BAciMK->ABciJK', H.a.oo[Oa, Oa], T.aab.VVvoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('mK,BAciJm->ABciJK', H.b.oo[ob, Ob], T.aab.VVvoOo, optimize=True)
        +1.0*np.einsum('MK,BAciJM->ABciJK', H.b.oo[Ob, Ob], T.aab.VVvoOO, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('Ae,BeciJK->ABciJK', H.a.vv[Va, va], T.aab.VvvoOO, optimize=True)
        -1.0*np.einsum('AE,BEciJK->ABciJK', H.a.vv[Va, Va], T.aab.VVvoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('ce,BAeiJK->ABciJK', H.b.vv[vb, vb], T.aab.VVvoOO, optimize=True)
        -1.0*np.einsum('cE,BAEiJK->ABciJK', H.b.vv[vb, Vb], T.aab.VVVoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -0.5*np.einsum('mniJ,BAcmnK->ABciJK', H.aa.oooo[oa, oa, oa, Oa], T.aab.VVvooO, optimize=True)
        -1.0*np.einsum('mNiJ,BAcmNK->ABciJK', H.aa.oooo[oa, Oa, oa, Oa], T.aab.VVvoOO, optimize=True)
        -0.5*np.einsum('MNiJ,BAcMNK->ABciJK', H.aa.oooo[Oa, Oa, oa, Oa], T.aab.VVvOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('MnJK,BAciMn->ABciJK', H.ab.oooo[Oa, ob, Oa, Ob], T.aab.VVvoOo, optimize=True)
        -1.0*np.einsum('mNJK,BAcimN->ABciJK', H.ab.oooo[oa, Ob, Oa, Ob], T.aab.VVvooO, optimize=True)
        -1.0*np.einsum('MNJK,BAciMN->ABciJK', H.ab.oooo[Oa, Ob, Oa, Ob], T.aab.VVvoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('mniK,BAcmJn->ABciJK', H.ab.oooo[oa, ob, oa, Ob], T.aab.VVvoOo, optimize=True)
        +1.0*np.einsum('MniK,BAcJMn->ABciJK', H.ab.oooo[Oa, ob, oa, Ob], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('mNiK,BAcmJN->ABciJK', H.ab.oooo[oa, Ob, oa, Ob], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('MNiK,BAcJMN->ABciJK', H.ab.oooo[Oa, Ob, oa, Ob], T.aab.VVvOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('ABeF,FeciJK->ABciJK', H.aa.vvvv[Va, Va, va, Va], T.aab.VvvoOO, optimize=True)
        -0.5*np.einsum('ABEF,FEciJK->ABciJK', H.aa.vvvv[Va, Va, Va, Va], T.aab.VVvoOO, optimize=True)
)
x3b += (2.0/2.0) * (
        +1.0*np.einsum('Bcef,AefiJK->ABciJK', H.ab.vvvv[Va, vb, va, vb], T.aab.VvvoOO, optimize=True)
        -1.0*np.einsum('BcEf,EAfiJK->ABciJK', H.ab.vvvv[Va, vb, Va, vb], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('BceF,AeFiJK->ABciJK', H.ab.vvvv[Va, vb, va, Vb], T.aab.VvVoOO, optimize=True)
        -1.0*np.einsum('BcEF,EAFiJK->ABciJK', H.ab.vvvv[Va, vb, Va, Vb], T.aab.VVVoOO, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('Amie,BecmJK->ABciJK', H.aa.voov[Va, oa, oa, va], T.aab.VvvoOO, optimize=True)
        -1.0*np.einsum('AmiE,BEcmJK->ABciJK', H.aa.voov[Va, oa, oa, Va], T.aab.VVvoOO, optimize=True)
        -1.0*np.einsum('AMie,BecMJK->ABciJK', H.aa.voov[Va, Oa, oa, va], T.aab.VvvOOO, optimize=True)
        -1.0*np.einsum('AMiE,BEcMJK->ABciJK', H.aa.voov[Va, Oa, oa, Va], T.aab.VVvOOO, optimize=True)
)
x3b += (2.0/2.0) * (
        +1.0*np.einsum('AmJe,BecmiK->ABciJK', H.aa.voov[Va, oa, Oa, va], T.aab.VvvooO, optimize=True)
        +1.0*np.einsum('AmJE,BEcmiK->ABciJK', H.aa.voov[Va, oa, Oa, Va], T.aab.VVvooO, optimize=True)
        -1.0*np.einsum('AMJe,BeciMK->ABciJK', H.aa.voov[Va, Oa, Oa, va], T.aab.VvvoOO, optimize=True)
        -1.0*np.einsum('AMJE,BEciMK->ABciJK', H.aa.voov[Va, Oa, Oa, Va], T.aab.VVvoOO, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('Amie,BceJmK->ABciJK', H.ab.voov[Va, ob, oa, vb], T.abb.VvvOoO, optimize=True)
        +1.0*np.einsum('AmiE,BEcJmK->ABciJK', H.ab.voov[Va, ob, oa, Vb], T.abb.VVvOoO, optimize=True)
        -1.0*np.einsum('AMie,BceJMK->ABciJK', H.ab.voov[Va, Ob, oa, vb], T.abb.VvvOOO, optimize=True)
        +1.0*np.einsum('AMiE,BEcJMK->ABciJK', H.ab.voov[Va, Ob, oa, Vb], T.abb.VVvOOO, optimize=True)
)
x3b += (2.0/2.0) * (
        +1.0*np.einsum('AmJe,BceimK->ABciJK', H.ab.voov[Va, ob, Oa, vb], T.abb.VvvooO, optimize=True)
        -1.0*np.einsum('AmJE,BEcimK->ABciJK', H.ab.voov[Va, ob, Oa, Vb], T.abb.VVvooO, optimize=True)
        +1.0*np.einsum('AMJe,BceiMK->ABciJK', H.ab.voov[Va, Ob, Oa, vb], T.abb.VvvoOO, optimize=True)
        -1.0*np.einsum('AMJE,BEciMK->ABciJK', H.ab.voov[Va, Ob, Oa, Vb], T.abb.VVvoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('mceK,BAeimJ->ABciJK', H.ab.ovvo[oa, vb, va, Ob], T.aaa.VVvooO, optimize=True)
        +1.0*np.einsum('mcEK,EBAimJ->ABciJK', H.ab.ovvo[oa, vb, Va, Ob], T.aaa.VVVooO, optimize=True)
        -1.0*np.einsum('MceK,BAeiJM->ABciJK', H.ab.ovvo[Oa, vb, va, Ob], T.aaa.VVvoOO, optimize=True)
        -1.0*np.einsum('McEK,EBAiJM->ABciJK', H.ab.ovvo[Oa, vb, Va, Ob], T.aaa.VVVoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('cmKe,BAeiJm->ABciJK', H.bb.voov[vb, ob, Ob, vb], T.aab.VVvoOo, optimize=True)
        -1.0*np.einsum('cmKE,BAEiJm->ABciJK', H.bb.voov[vb, ob, Ob, Vb], T.aab.VVVoOo, optimize=True)
        -1.0*np.einsum('cMKe,BAeiJM->ABciJK', H.bb.voov[vb, Ob, Ob, vb], T.aab.VVvoOO, optimize=True)
        -1.0*np.einsum('cMKE,BAEiJM->ABciJK', H.bb.voov[vb, Ob, Ob, Vb], T.aab.VVVoOO, optimize=True)
)
x3b += (2.0/2.0) * (
        +1.0*np.einsum('AmeK,BeciJm->ABciJK', H.ab.vovo[Va, ob, va, Ob], T.aab.VvvoOo, optimize=True)
        +1.0*np.einsum('AmEK,BEciJm->ABciJK', H.ab.vovo[Va, ob, Va, Ob], T.aab.VVvoOo, optimize=True)
        +1.0*np.einsum('AMeK,BeciJM->ABciJK', H.ab.vovo[Va, Ob, va, Ob], T.aab.VvvoOO, optimize=True)
        +1.0*np.einsum('AMEK,BEciJM->ABciJK', H.ab.vovo[Va, Ob, Va, Ob], T.aab.VVvoOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('mcie,BAemJK->ABciJK', H.ab.ovov[oa, vb, oa, vb], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('mciE,BAEmJK->ABciJK', H.ab.ovov[oa, vb, oa, Vb], T.aab.VVVoOO, optimize=True)
        +1.0*np.einsum('Mcie,BAeMJK->ABciJK', H.ab.ovov[Oa, vb, oa, vb], T.aab.VVvOOO, optimize=True)
        +1.0*np.einsum('MciE,BAEMJK->ABciJK', H.ab.ovov[Oa, vb, oa, Vb], T.aab.VVVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('mcJe,BAemiK->ABciJK', H.ab.ovov[oa, vb, Oa, vb], T.aab.VVvooO, optimize=True)
        -1.0*np.einsum('mcJE,BAEmiK->ABciJK', H.ab.ovov[oa, vb, Oa, Vb], T.aab.VVVooO, optimize=True)
        +1.0*np.einsum('McJe,BAeiMK->ABciJK', H.ab.ovov[Oa, vb, Oa, vb], T.aab.VVvoOO, optimize=True)
        +1.0*np.einsum('McJE,BAEiMK->ABciJK', H.ab.ovov[Oa, vb, Oa, Vb], T.aab.VVVoOO, optimize=True)
)

x3b -= np.transpose(x3b, (1, 0, 2, 3, 4, 5))

get_error(X3B[Va, Va, va, oa, Oa, Ob], x3b)

2.3528447148704464e-17


### Projection 9

In [19]:
x3b = (2.0/2.0) * (
        -1.0*np.einsum('mI,AbCmJk->AbCIJk', H.a.oo[oa, Oa], T.aab.VvVoOo, optimize=True)
        -1.0*np.einsum('MI,AbCMJk->AbCIJk', H.a.oo[Oa, Oa], T.aab.VvVOOo, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('mk,AbCIJm->AbCIJk', H.b.oo[ob, ob], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('Mk,AbCIJM->AbCIJk', H.b.oo[Ob, ob], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('Ae,beCIJk->AbCIJk', H.a.vv[Va, va], T.aab.vvVOOo, optimize=True)
        +1.0*np.einsum('AE,EbCIJk->AbCIJk', H.a.vv[Va, Va], T.aab.VvVOOo, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('be,AeCIJk->AbCIJk', H.a.vv[va, va], T.aab.VvVOOo, optimize=True)
        +1.0*np.einsum('bE,AECIJk->AbCIJk', H.a.vv[va, Va], T.aab.VVVOOo, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('Ce,AbeIJk->AbCIJk', H.b.vv[Vb, vb], T.aab.VvvOOo, optimize=True)
        +1.0*np.einsum('CE,AbEIJk->AbCIJk', H.b.vv[Vb, Vb], T.aab.VvVOOo, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('mNIJ,AbCmNk->AbCIJk', H.aa.oooo[oa, Oa, Oa, Oa], T.aab.VvVoOo, optimize=True)
        +0.5*np.einsum('MNIJ,AbCMNk->AbCIJk', H.aa.oooo[Oa, Oa, Oa, Oa], T.aab.VvVOOo, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('mnJk,AbCmIn->AbCIJk', H.ab.oooo[oa, ob, Oa, ob], T.aab.VvVoOo, optimize=True)
        +1.0*np.einsum('MnJk,AbCIMn->AbCIJk', H.ab.oooo[Oa, ob, Oa, ob], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('mNJk,AbCmIN->AbCIJk', H.ab.oooo[oa, Ob, Oa, ob], T.aab.VvVoOO, optimize=True)
        +1.0*np.einsum('MNJk,AbCIMN->AbCIJk', H.ab.oooo[Oa, Ob, Oa, ob], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -0.5*np.einsum('Abef,feCIJk->AbCIJk', H.aa.vvvv[Va, va, va, va], T.aab.vvVOOo, optimize=True)
        +1.0*np.einsum('AbEf,EfCIJk->AbCIJk', H.aa.vvvv[Va, va, Va, va], T.aab.VvVOOo, optimize=True)
        -0.5*np.einsum('AbEF,FECIJk->AbCIJk', H.aa.vvvv[Va, va, Va, Va], T.aab.VVVOOo, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('bCef,AefIJk->AbCIJk', H.ab.vvvv[va, Vb, va, vb], T.aab.VvvOOo, optimize=True)
        +1.0*np.einsum('bCeF,AeFIJk->AbCIJk', H.ab.vvvv[va, Vb, va, Vb], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('bCEf,EAfIJk->AbCIJk', H.ab.vvvv[va, Vb, Va, vb], T.aab.VVvOOo, optimize=True)
        -1.0*np.einsum('bCEF,EAFIJk->AbCIJk', H.ab.vvvv[va, Vb, Va, Vb], T.aab.VVVOOo, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('ACeF,ebFIJk->AbCIJk', H.ab.vvvv[Va, Vb, va, Vb], T.aab.vvVOOo, optimize=True)
        +1.0*np.einsum('ACEf,EbfIJk->AbCIJk', H.ab.vvvv[Va, Vb, Va, vb], T.aab.VvvOOo, optimize=True)
        +1.0*np.einsum('ACEF,EbFIJk->AbCIJk', H.ab.vvvv[Va, Vb, Va, Vb], T.aab.VvVOOo, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('AmIe,beCmJk->AbCIJk', H.aa.voov[Va, oa, Oa, va], T.aab.vvVoOo, optimize=True)
        -1.0*np.einsum('AMIe,beCMJk->AbCIJk', H.aa.voov[Va, Oa, Oa, va], T.aab.vvVOOo, optimize=True)
        +1.0*np.einsum('AmIE,EbCmJk->AbCIJk', H.aa.voov[Va, oa, Oa, Va], T.aab.VvVoOo, optimize=True)
        +1.0*np.einsum('AMIE,EbCMJk->AbCIJk', H.aa.voov[Va, Oa, Oa, Va], T.aab.VvVOOo, optimize=True)
)
x3b += (2.0/2.0) * (
        +1.0*np.einsum('bmIe,AeCmJk->AbCIJk', H.aa.voov[va, oa, Oa, va], T.aab.VvVoOo, optimize=True)
        +1.0*np.einsum('bMIe,AeCMJk->AbCIJk', H.aa.voov[va, Oa, Oa, va], T.aab.VvVOOo, optimize=True)
        +1.0*np.einsum('bmIE,AECmJk->AbCIJk', H.aa.voov[va, oa, Oa, Va], T.aab.VVVoOo, optimize=True)
        +1.0*np.einsum('bMIE,AECMJk->AbCIJk', H.aa.voov[va, Oa, Oa, Va], T.aab.VVVOOo, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('AmIe,bCeJmk->AbCIJk', H.ab.voov[Va, ob, Oa, vb], T.abb.vVvOoo, optimize=True)
        +1.0*np.einsum('AMIe,bCeJkM->AbCIJk', H.ab.voov[Va, Ob, Oa, vb], T.abb.vVvOoO, optimize=True)
        -1.0*np.einsum('AmIE,bCEJmk->AbCIJk', H.ab.voov[Va, ob, Oa, Vb], T.abb.vVVOoo, optimize=True)
        +1.0*np.einsum('AMIE,bCEJkM->AbCIJk', H.ab.voov[Va, Ob, Oa, Vb], T.abb.vVVOoO, optimize=True)
)
x3b += (2.0/2.0) * (
        +1.0*np.einsum('bmIe,ACeJmk->AbCIJk', H.ab.voov[va, ob, Oa, vb], T.abb.VVvOoo, optimize=True)
        -1.0*np.einsum('bMIe,ACeJkM->AbCIJk', H.ab.voov[va, Ob, Oa, vb], T.abb.VVvOoO, optimize=True)
        +1.0*np.einsum('bmIE,ACEJmk->AbCIJk', H.ab.voov[va, ob, Oa, Vb], T.abb.VVVOoo, optimize=True)
        -1.0*np.einsum('bMIE,ACEJkM->AbCIJk', H.ab.voov[va, Ob, Oa, Vb], T.abb.VVVOoO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('mCek,AebmIJ->AbCIJk', H.ab.ovvo[oa, Vb, va, ob], T.aaa.VvvoOO, optimize=True)
        -1.0*np.einsum('MCek,AebIJM->AbCIJk', H.ab.ovvo[Oa, Vb, va, ob], T.aaa.VvvOOO, optimize=True)
        +1.0*np.einsum('mCEk,EAbmIJ->AbCIJk', H.ab.ovvo[oa, Vb, Va, ob], T.aaa.VVvoOO, optimize=True)
        +1.0*np.einsum('MCEk,EAbIJM->AbCIJk', H.ab.ovvo[Oa, Vb, Va, ob], T.aaa.VVvOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('Cmke,AbeIJm->AbCIJk', H.bb.voov[Vb, ob, ob, vb], T.aab.VvvOOo, optimize=True)
        +1.0*np.einsum('CMke,AbeIJM->AbCIJk', H.bb.voov[Vb, Ob, ob, vb], T.aab.VvvOOO, optimize=True)
        +1.0*np.einsum('CmkE,AbEIJm->AbCIJk', H.bb.voov[Vb, ob, ob, Vb], T.aab.VvVOOo, optimize=True)
        +1.0*np.einsum('CMkE,AbEIJM->AbCIJk', H.bb.voov[Vb, Ob, ob, Vb], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        +1.0*np.einsum('Amek,beCIJm->AbCIJk', H.ab.vovo[Va, ob, va, ob], T.aab.vvVOOo, optimize=True)
        +1.0*np.einsum('AMek,beCIJM->AbCIJk', H.ab.vovo[Va, Ob, va, ob], T.aab.vvVOOO, optimize=True)
        -1.0*np.einsum('AmEk,EbCIJm->AbCIJk', H.ab.vovo[Va, ob, Va, ob], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('AMEk,EbCIJM->AbCIJk', H.ab.vovo[Va, Ob, Va, ob], T.aab.VvVOOO, optimize=True)
)
x3b += (1.0/2.0) * (
        -1.0*np.einsum('bmek,AeCIJm->AbCIJk', H.ab.vovo[va, ob, va, ob], T.aab.VvVOOo, optimize=True)
        -1.0*np.einsum('bMek,AeCIJM->AbCIJk', H.ab.vovo[va, Ob, va, ob], T.aab.VvVOOO, optimize=True)
        -1.0*np.einsum('bmEk,AECIJm->AbCIJk', H.ab.vovo[va, ob, Va, ob], T.aab.VVVOOo, optimize=True)
        -1.0*np.einsum('bMEk,AECIJM->AbCIJk', H.ab.vovo[va, Ob, Va, ob], T.aab.VVVOOO, optimize=True)
)
x3b += (2.0/2.0) * (
        -1.0*np.einsum('mCIe,AbemJk->AbCIJk', H.ab.ovov[oa, Vb, Oa, vb], T.aab.VvvoOo, optimize=True)
        -1.0*np.einsum('MCIe,AbeMJk->AbCIJk', H.ab.ovov[Oa, Vb, Oa, vb], T.aab.VvvOOo, optimize=True)
        -1.0*np.einsum('mCIE,AbEmJk->AbCIJk', H.ab.ovov[oa, Vb, Oa, Vb], T.aab.VvVoOo, optimize=True)
        -1.0*np.einsum('MCIE,AbEMJk->AbCIJk', H.ab.ovov[Oa, Vb, Oa, Vb], T.aab.VvVOOo, optimize=True)
)

x3b -= np.transpose(x3b, (0, 1, 2, 4, 3, 5))

get_error(X3B[Va, va, Vb, Oa, Oa, ob], x3b)

5.371258975555742e-17
